In [21]:
import pandas as pd
import numpy as np


# Определение ставок VAT и duty
vat_rates = {
    'DE': 0.19,            # Germany
    'PT': 0.23,            # Portugal (mainland, Madeira, Azores)
    'ES': 0.21,            # Spain (except Canary Islands)
    'IE': 0.23,            # Ireland
    'SE': 0.25,            # Sweden
    'NL': 0.21,            # Netherlands
    'DK': 0.25,            # Denmark (mainland only)
    'FI': 0.255,           # Finland (except Åland Islands, raised in 2024)
    'IT': 0.22,            # Italy (except Livigno, Campione d’Italia, Lake Lugano waters)
    'AT': 0.20,            # Austria
    'BE': 0.21,            # Belgium
    'EE': 0.22             # Estonia (until July); use 0.24 for returns post-July 2025
}

In [22]:
df = pd.read_csv("TBdata.csv")

df = df[~df['Consignee Country'].isin(['IC', 'CH'])] # exclude all the non-eu countries

#fill all the missing MRN-values with Parcel ID:
df['MRN'] = df['MRN'].fillna(df['Parcel ID'])

# Ensure correct consignment grouping
df['Consignment Value'] = df.groupby('MRN')['Line Item Unit Price'].transform('sum')

df = df[df['Consignment Value'] <= 150]

df = df[df['Line Item Quantity Returned'] > 0] # оставляем только те товары, которые вернули

In [23]:
df['VAT Rate'] = df['Consignee Country'].map(vat_rates)

df['VAT Returned'] = df['Line Item Quantity Returned'] * df['Line Item Unit Price'] * df['VAT Rate']

In [24]:
# group and sum specific columns
df = df.groupby(['Consignee Country', 'VAT Rate'], as_index=False).agg({
    'VAT Returned': 'sum'
})

# apply 0.3% for IE, otherwise 0.2%
df['DR Revenue'] = df['VAT Returned'] * np.where(df['Consignee Country'] == 'IE', 0.3, 0.2)

print(df[['Consignee Country','VAT Rate', 'VAT Returned' ,'DR Revenue']])

print(f'\nOUR REVENUE: {df['DR Revenue'].sum()}')

  Consignee Country  VAT Rate  VAT Returned  DR Revenue
0                AT     0.200        42.600       8.520
1                DE     0.190       611.040     122.208
2                FI     0.255        65.535      13.107
3                IE     0.230       883.430     265.029
4                NL     0.210       160.860      32.172

OUR REVENUE: 441.03600000000006
